In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Mary\AppData\Local\Temp\ipykernel_5620\2151744951.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Data preparation

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

df = pd.read_csv("../Data/Advertising.csv", index_col = 0)
X, y = df.drop("Sales", axis = 1), df["Sales"]

model_polynomial = PolynomialFeatures(3, include_bias = False)
poly_features = model_polynomial.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size = 0.33, random_state = 42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 19), (66, 19), (134,), (66,))

## Feature standardization

LASSO, Ridge and Elasticnet regressions require that the data is scaled.

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

## Ridge regression (linear regression with L2 regularization)

In [4]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

def ridge_regression(X, penalty = 0):
    model_ridge = Ridge(alpha = penalty)
    model_ridge.fit(scaled_X_train, y_train)
    y_pred = model_ridge.predict(X)
    return y_pred

y_pred = ridge_regression(scaled_X_test, penalty = 0.2)
MSE = mean_squared_error (y_test, y_pred)
RMSE = np.sqrt (MSE)

RMSE, mean_absolute_error(y_test, y_pred)

(0.6109310380379477, 0.4845959994544089)

In [5]:
# comparing with linear regression (similar RMSE, the same when lambda = 0)

from sklearn.linear_model import LinearRegression
model_linear = LinearRegression()
model_linear = LinearRegression()
model_linear.fit(scaled_X_train, y_train)
y_pred = model_linear.predict(scaled_X_test)
np.sqrt (mean_squared_error (y_test, y_pred)), mean_absolute_error (y_test, y_pred)


(0.5148267621786614, 0.3748516441217832)

In [13]:
model_linear.coef_

array([  7.78963284,   0.16524713,   0.58180175, -11.35122755,
         4.77455328,  -1.61094033,   0.07752703,   0.24197855,
         0.1384483 ,   5.28808537,  -0.78605153,   1.39665099,
        -0.03107333,  -0.5613837 ,   0.05225192,  -0.04142188,
         0.07302452,  -0.03754884,  -0.21894604])

## Lasso regression (Linear regression + L1 reg)

In [6]:
from sklearn.linear_model import Lasso
model_Lasso = Lasso (alpha = .1)
model_Lasso.fit(scaled_X_train, y_train)
y_pred = model_Lasso.predict(scaled_X_test)
np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

(0.7853962108799017, 0.5735346450114956)

## K-fold cross-validation

* with Ridge regression

In [8]:
from sklearn.linear_model import RidgeCV # ridge regression with cross-validation
import sklearn.metrics as metrics


In [10]:
model_ridgeCV = RidgeCV(alphas = [.0001, .001, .01, .1, .5, 1, 5, 10], scoring = "neg_mean_squared_error")
model_ridgeCV.fit(scaled_X_train, y_train)
print(model_ridgeCV.alpha_)

0.1


In [11]:
y_pred = model_ridgeCV.predict(scaled_X_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE, mean_absolute_error(y_test, y_pred)

(0.5635899169599432, 0.4343075766531668)

In [14]:
model_ridgeCV.coef_

array([ 5.84681185,  0.52142086,  0.71689997, -6.17948738,  3.75034058,
       -1.36283352, -0.08571128,  0.08322815, -0.34893776,  2.16952446,
       -0.47840838,  0.68527348,  0.63080799, -0.5950065 ,  0.61661989,
       -0.31335495,  0.36499629,  0.03328145, -0.13652471])

* with Lasso regression

In [15]:
from sklearn.linear_model import LassoCV

In [16]:
model_lassoCV = LassoCV(eps = 0.001, n_alphas = 100, max_iter = 10000, cv = 5)

* eps - параметр, который определяет критерий остановки для итерационного процесса оптимизации. Алгоритм Lasso будет прекращать итерации, если изменение в весах (коэффициентах) становится меньше eps.

* n_alphas: Количество значений коэффициента регуляризации (альфа), которые будут протестированы в процессе кросс-валидации. В данном случае, 100 различных значений.

* max_iter: Максимальное количество итераций для оптимизационного алгоритма. Если алгоритм не сходится к оптимальному решению в пределах этого количества итераций, он прекращает работу.

In [17]:
model_lassoCV.fit(scaled_X_train, y_train)
model_lassoCV.alpha_

0.004968802520343365

In [18]:
y_pred = model_lassoCV.predict(scaled_X_test)
np.sqrt(mean_squared_error(y_test,y_pred)), mean_absolute_error(y_test, y_pred)

(0.5785146895301981, 0.4629188302693299)

In [19]:
model_lassoCV.coef_

array([ 5.19612354,  0.43037087,  0.29876351, -4.80417579,  3.46665205,
       -0.40507212,  0.        ,  0.        ,  0.        ,  1.35260206,
       -0.        ,  0.        ,  0.14879719, -0.        ,  0.        ,
        0.        ,  0.09649665,  0.        ,  0.04353956])

## Elastic net

In [20]:
from sklearn.linear_model import ElasticNetCV

In [23]:
model_elastic = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], eps = 0.001, n_alphas = 100, max_iter = 10000, cv = 5)
model_elastic.fit(scaled_X_train, y_train)
print(f"L1 ratio: {model_elastic.l1_ratio_}, lambda: {model_elastic.alpha_}")

L1 ratio: 1.0, lambda: 0.004968802520343365


In [24]:
y_pred = model_elastic.predict(scaled_X_test)
np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

(0.5785146895301981, 0.4629188302693299)